# Project 1

Team 7:
- Sai Prasath Kuppusamy Jayasree
- Arun Arunachalam Maheswaran

### Introduction

The goal of the project is to understand the data science project workflow from scratch and going through a typical process from Reading, Preprocessing, Analyzing and Model Fitting and Prediction. The project also emphasises on good programming practices which will be necessary in the real world scenario for code readability and understanding. 


The given data set, containing enrollment data for different areas, is accessed through a specified URL in .csv format. The data is transformed from the original wide format into a long format and then separated into county and non-county data through processing. A wrapper function, along with multiple helper functions, is employed to read and process various data sets.

Next, the county level data set is utilized to fit both a simple linear regression model and a multiple linear regression model. To evaluate these models, cross-validation is used with the mean squared error (MSE) as the metric. Specifically, a function is employed to train the first three years of the data set and predict the value for the fourth year. This process is repeated for all years in the data set and the cumulative MSE is calculated and compared for model evaluation.


## Part 1: Data Processing

### Task 1: Read in the data

The given data set is in **.csv** format and we will be using pandas module to read and parse through the data set.

In [1]:
#importing required modules
import pandas as pd
import numpy as np
from functools import reduce
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

#reading the first part of the data set
survey_data = pd.read_csv("https://www4.stat.ncsu.edu/~online/datasets/EDU01a.csv")
survey_data.head()

,Area_name,STCOU,EDU010187F,EDU010187D,EDU010187N1,EDU010187N2,EDU010188F,EDU010188D,EDU010188N1,EDU010188N2,...,EDU010194N1,EDU010194N2,EDU010195F,EDU010195D,EDU010195N1,EDU010195N2,EDU010196F,EDU010196D,EDU010196N1,EDU010196N2
0,UNITED STATES,0,0,40024299,0,0,0,39967624,0,0,...,0,0,0,43993459,0,0,0,44715737,0,0
1,ALABAMA,1000,0,733735,0,0,0,728234,0,0,...,0,0,0,727989,0,0,0,736825,0,0
2,"Autauga, AL",1001,0,6829,0,0,0,6900,0,0,...,0,0,0,7568,0,0,0,7834,0,0
3,"Baldwin, AL",1003,0,16417,0,0,0,16465,0,0,...,0,0,0,19961,0,0,0,20699,0,0
4,"Barbour, AL",1005,0,5071,0,0,0,5098,0,0,...,0,0,0,5017,0,0,0,5053,0,0


The dataset consists of 42 columns where **Area_name** contains *county* and *non county* data, **STCOU** column contains the county code. All the other columns represent enrollment data for different years.

In [2]:
#selecting the columns that only ends with 'D'
sub_data = survey_data[["Area_name", "STCOU"] + [x for x in survey_data.loc[:,[x for x in survey_data.columns if x.endswith('D')]].columns]]

#converting the wide format data into long format
sub_data = sub_data.melt(id_vars = ["Area_name", "STCOU"], var_name= "info", value_name= "enrollment")
sub_data.head()

,Area_name,STCOU,info,enrollment
0,UNITED STATES,0,EDU010187D,40024299
1,ALABAMA,1000,EDU010187D,733735
2,"Autauga, AL",1001,EDU010187D,6829
3,"Baldwin, AL",1003,EDU010187D,16417
4,"Barbour, AL",1005,EDU010187D,5071


The given data set is converted from **wide** format into **long** format by stacking the columns that ends with **D** as a row value of the column *info* (corresponds to particular year's measurements) and the values of these original columns are passed into the new column *enrollment*.

In [3]:
#extracting the 'year' and 'measurement' data from 'info' column
for i in range(0, len(sub_data)):
    if sub_data["info"][i][-4] == "1":
        sub_data.loc[i,"year"] = int("19" + sub_data["info"][i][-3:-1])
    elif sub_data["info"][i][-4] == "2":
        sub_data.loc[i,"year"] = int("20" + sub_data["info"][i][-3:-1])
    sub_data.loc[i,"measurement"] = sub_data["info"][i][:7]
sub_data = sub_data.drop("info", axis = 1)

The *year* value is extracted from the *info* column values. If the **7th element** of *info* value is 1 then the year starts with **19** and if it is 2 then the year starts with **20**. The last two digits of the year will be **8th and 9th element** of *info*

In [4]:
#splitting the data frame into county and non county data
county_data = sub_data[sub_data["Area_name"].apply(lambda x: ',' in x)]
nonCounty_data = sub_data[np.logical_not(sub_data["Area_name"].apply(lambda x: ',' in x))]

county_data["state"] = county_data["Area_name"].str[-2:]

/tmp/ipykernel_12560/1003401584.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_data["state"] = county_data["Area_name"].str[-2:]


In the data set, if the *Area_name* column value has a '**,**' (comma), then the row entry is considered as *County* data. Otherwise, the entry is considered as *Non county* data.

In [5]:
def div_func(x):    
    '''
    Assign division number for corresponding state

    Parameters
    ----------
    x: str
    name of the state

    Returns
    -------
    'Division #': str
    corresponding division of the state
   '''
    if x.title() in ["Connecticut", "Maine", "Massachusetts", "New Hampshire", "Rhode Island", "Vermont"]:
        return "Division 1"
    elif x.title() in ["New Jersey", "New York", "Pennsylvania"]:
        return "Division 2"
    elif x.title() in ["Illinois", "Indiana", "Michigan", "Ohio", "Wisconsin"]:
        return "Division 3"
    elif x.title() in ["Iowa", "Kansas", "Minnesota", "Missouri", "Nebraska", "North Dakota", "South Dakota"]:
        return "Division 4"
    elif x.title() in ["Delaware", "Florida", "Georgia", "Maryland", "North Carolina", "South Carolina", "Virginia", "Washington", "D.C.", "West Virginia"]:
        return "Division 5"
    elif x.title() in ["Alabama", "Kentucky", "Mississippi", "Tennessee"]:
        return "Division 6"
    elif x.title() in ["Arkansas", "Louisiana", "Oklahoma", "Texas"]:
        return "Division 7"
    elif x.title() in ["Arizona", "Colorado", "Idaho", "Montana", "Nevada", "New Mexico", "Utah", "Wyoming"]:
        return "Division 8"
    elif x.title() in ["Alaska", "California", "Hawaii", "Oregon", "Washington"]:
        return "Division 9"
    else:
        return "ERROR"

vfunc = np.vectorize(div_func)
nonCounty_data["division"] = vfunc(nonCounty_data["Area_name"])

/tmp/ipykernel_12560/3817666394.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonCounty_data["division"] = vfunc(nonCounty_data["Area_name"])


In [6]:
survey_data2 = pd.read_csv("https://www4.stat.ncsu.edu/~online/datasets/EDU01b.csv")

def select_long(data, val = "enrollment"):
    '''
      Select the columns that only ends with 'D' and converting into long format

      Parameters
      ----------
      data: pandas DataFrame
        datframe with all columns and in wide format

      Returns
      -------
      data: pandas DataFrame
        dataframe with selected columns and in long format
    '''
    data_sub = data[["Area_name", "STCOU"] + [x for x in data.loc[:,[x for x in data.columns if x.endswith('D')]].columns]]
    data_sub = data_sub.melt(id_vars = ["Area_name", "STCOU"], var_name= "info", value_name= val)
    return data_sub

In [7]:
def year_measure(data):
    '''
      Extract the 'year' and 'measurement' data from 'info' column

      Parameters
      ----------
      data: pandas DataFrame
        Data frame in long format

      Returns
      -------
      data: pandas DataFrame
        Data frame with year and measurement column
    '''
    for i in range(0, len(data)):
        if data["info"][i][-4] == "1":
            data.loc[i,"year"] = int("19" + data["info"][i][-3:-1])
        elif data["info"][i][-4] == "2":
            data.loc[i,"year"] = int("20" + data["info"][i][-3:-1])
        data.loc[i,"measurement"] = data["info"][i][:7]
    data = data.drop("info", axis = 1)
    return data

In [8]:
def county_state(data):
    '''
      Filter the dataframe to get the county data

      Parameters
      ----------
      data: pandas DataFrame
        Data frame with only county data

      Returns
      -------
      data: pandas DataFrame
        Data frame with state column
    '''
    data["state"] = data["Area_name"].str[-2:]
    return data
    
def nonCounty_division(data):
    '''
      Filter the dataframe to get the non county data

      Parameters
      ----------
      data: pandas DataFrame
        Data frame with only non county data

      Returns
      -------
      data: pandas DataFrame
        Data frame with non county data with division #
    '''
    vfunc = np.vectorize(div_func)
    data["division"] = vfunc(data["Area_name"])
    return data

def final_step(data):
    '''
      splits the data to county and con county data and wraps county_state and nonCounty_division functions

      Parameters
      ----------
      data: pandas DataFrame
        Data frame with county and non county data

      Returns
      -------
      county_state(county_data): pandas DataFrame with county data with state column
      nonCounty_division(nonCounty_data): pandas DataFrame with non county data with division column
        Data frame with year and measurement column
    '''
    county_data = data[data["Area_name"].apply(lambda x: ',' in x)]
    nonCounty_data = data[np.logical_not(data["Area_name"].apply(lambda x: ',' in x))]
    return county_state(county_data), nonCounty_division(nonCounty_data)

In [9]:
def wrapper_func(url,val = "enrollment"):
    '''
      wraps all the helper function

      Parameters
      ----------
      url: url string

      Returns
      -------
      [data3, data4]: list of two pandas DataFrame
        Data frame with county and non county data
    '''
    data0 = pd.read_csv(url)
    data1 = select_long(data0, val)
    data2 = year_measure(data1)
    data3, data4 = final_step(data2)
    return [data3, data4]

In [10]:
#call the wrapper function to return the list of two data frames with county and non county data
mixData1 =wrapper_func("https://www4.stat.ncsu.edu/~online/datasets/EDU01a.csv", val = "enrollment")
mixData2 = wrapper_func("https://www4.stat.ncsu.edu/~online/datasets/EDU01b.csv", val = "enrollment")

/tmp/ipykernel_12560/815759403.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["state"] = data["Area_name"].str[-2:]
/tmp/ipykernel_12560/815759403.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["division"] = vfunc(data["Area_name"])
/tmp/ipykernel_12560/815759403.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

## Part 2: Combining data functions

In [11]:
def data_concat(*args):
    '''
      split the list of data frames and concatenate with corresponding county and non county data

      Parameters
      ----------
      *args: unlimited positional arguments
        two lists, each consisting of two data frames

      Returns
      -------
      [countyCat, nonCountyCat]: list of two pandas DataFrames
        list of data frame with concatenated county and non county data
    '''
    county = list(map(lambda x: x[0], args))
    countyCat = reduce(lambda x,y: pd.concat([x,y]), county)
    nonCounty = list(map(lambda x: x[1], args))
    nonCountyCat = reduce(lambda x,y: pd.concat([x,y]), nonCounty)
    return [countyCat, nonCountyCat]

In [12]:
#calling the function to check the result
data_concat(mixData1, mixData2)

[            Area_name  STCOU  enrollment    year measurement state
 2         Autauga, AL   1001        6829  1987.0     EDU0101    AL
 3         Baldwin, AL   1003       16417  1987.0     EDU0101    AL
 4         Barbour, AL   1005        5071  1987.0     EDU0101    AL
 5            Bibb, AL   1007        3557  1987.0     EDU0101    AL
 6          Blount, AL   1009        7319  1987.0     EDU0101    AL
 ...               ...    ...         ...     ...         ...   ...
 31975  Sweetwater, WY  56037        6964  2006.0     EDU0152    WY
 31976       Teton, WY  56039        2264  2006.0     EDU0152    WY
 31977       Uinta, WY  56041        4298  2006.0     EDU0152    WY
 31978    Washakie, WY  56043        1410  2006.0     EDU0152    WY
 31979      Weston, WY  56045        1076  2006.0     EDU0152    WY
 
 [62900 rows x 6 columns],
            Area_name  STCOU  enrollment    year measurement    division
 0      UNITED STATES      0    40024299  1987.0     EDU0101       ERROR
 1       

From the result above, we could see that the **county data** and the **non county data** from two urls mixData1 and mixData2 are concatenated and returned as a list of two concatenated data frames.

In [13]:
#double checking the functions for four urls
mixData3 =wrapper_func("https://www4.stat.ncsu.edu/~online/datasets/PST01a.csv", val = "enrollment")
mixData4 = wrapper_func("https://www4.stat.ncsu.edu/~online/datasets/PST01b.csv", val = "enrollment")
mixData5 =wrapper_func("https://www4.stat.ncsu.edu/~online/datasets/PST01c.csv", val = "enrollment")
mixData6 = wrapper_func("https://www4.stat.ncsu.edu/~online/datasets/PST01d.csv", val = "enrollment")
data_concat(mixData3, mixData4, mixData5, mixData6)

/tmp/ipykernel_12560/815759403.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["state"] = data["Area_name"].str[-2:]
/tmp/ipykernel_12560/815759403.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["division"] = vfunc(data["Area_name"])
/tmp/ipykernel_12560/815759403.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

[            Area_name  STCOU  enrollment    year measurement state
 2         Autauga, AL   1001       25508  1971.0     PST0151    AL
 3         Baldwin, AL   1003       60141  1971.0     PST0151    AL
 4         Barbour, AL   1005       23092  1971.0     PST0151    AL
 5            Bibb, AL   1007       13919  1971.0     PST0151    AL
 6          Blount, AL   1009       27817  1971.0     PST0151    AL
 ...               ...    ...         ...     ...         ...   ...
 31975  Sweetwater, WY  56037       41226  2009.0     PST0452    WY
 31976       Teton, WY  56039       20710  2009.0     PST0452    WY
 31977       Uinta, WY  56041       20927  2009.0     PST0452    WY
 31978    Washakie, WY  56043        7911  2009.0     PST0452    WY
 31979      Weston, WY  56045        7009  2009.0     PST0452    WY
 
 [125800 rows x 6 columns],
            Area_name  STCOU  enrollment    year measurement    division
 0      UNITED STATES      0   206827028  1971.0     PST0151       ERROR
 1      

## Part 3: Cross validation functions

In [14]:
#subset the data
nC_data1 = mixData1[1]
nC_data1 = nC_data1[nC_data1["division"] != "ERROR"]
nC_data1 = nC_data1[["year", "division", "enrollment"]]

In [15]:
#creating dummies for the division values
nC_dummies = pd.get_dummies(data = nC_data1["division"])

In [16]:
#adding the dummy column to the subsetted data frame
for i in nC_dummies.columns:
    nC_data1[i] = nC_dummies[i].copy()

In [17]:
#drop the division and division 9 columns
nC_data1 = nC_data1.drop(["division", "Division 9"], axis = 1)

In [18]:
#checking the new columns of the data frame
nC_data1.head()

,year,enrollment,Division 1,Division 2,Division 3,Division 4,Division 5,Division 6,Division 7,Division 8
1,1987.0,733735,0,0,0,0,0,1,0,0
69,1987.0,102872,0,0,0,0,0,0,0,0
99,1987.0,609411,0,0,0,0,0,0,0,1
115,1987.0,429260,0,0,0,0,0,0,1,0
191,1987.0,4621126,0,0,0,0,0,0,0,0


We first write a function which we can use to get one MSE values for any given year and the model. This function can be used in the next step where we loop through different years and find MSE for all those years

In [19]:
def oneStep_mse(x, y, last_year, model):
    """
    Function to get MSE value for a particular year after fitting a linear regression model between the predictor variables
    on 2 different models
    
    Parameters
    ----------
    x: Pandas Dataframe or Series
     Predictor variables which is usually the year or year and divison based on the models
    y: Pandas Series
     Response variables which are dependent on the independent variables x, in this case its the enrollment number
    last_year: int
     Year we use to split between test and train set
    model: str
     linear model we want to fit. slr for single linear regression and mlr for multivariate linear regression
     
    
    Returns
    --------
    test_MSE: float
     Mean Squared Error for the year we want to predict through previous years
    
    """
    #test train data split
    X_train = x[x["year"] <= last_year]
    X_test = x[x["year"] == last_year + 1]
    
    y_train = y.filter(items = X_train.index, axis = 0)
    y_test = y.filter(items = X_test.index, axis = 0)
    
    #conditional statements to find mse for given model
    if model == 'slr':
        reg = linear_model.LinearRegression()
        reg.fit(X = X_train["year"].values.reshape(-1,1), y = y_train)
        test_MSE = mean_squared_error(y_true = y_test.values, y_pred = reg.predict(X_test["year"].values.reshape(-1,1)))
    elif model == 'mlr':
        reg_mlr = linear_model.LinearRegression()
        reg_mlr.fit(X = X_train, y = y_train)
        test_MSE = mean_squared_error(y_true = y_test.values, y_pred = reg_mlr.predict(X_test))
    
    return test_MSE

This function will be used to find the CV value over all the years given the training model. We raise an exception whenever we get an inapporopriate start year. If we get the right start year, we loop through the years and finding the cumulative mse error for each year and put in in a dictionary for future use

In [20]:
def obtainCV(x, y, first_year, model):
    """
    Function to obtain Crossvalidation error for different linear regression models across all the years
    
    Parameters
    ----------
    x: Pandas Dataframe or Series
     Predictor variables which is usually the year or year and divison based on the models
    y: Pandas Series
     Response variables which are dependent on the independent variables x, in this case its the enrollment number
    first_year: int
     Year from which we want to start collecting MSE values for.
    model: str
     linear model we want to fit. slr for single linear regression and mlr for multivariate linear regression
     
    
    Returns
    --------
    mse: float
     cummulative mean squared error value for the last year
    mse_dict: dictionary
     dictionary of cummulative mean squared errors for different last years
    
    """
    
    if first_year < 1989:
        raise Exception('first_year is less than 1989')
    mse = 0
    mse_dict = {}
    last_year = int(x['year'].max())
    for i in range(first_year,last_year):  
        mse += oneStep_mse(x, y, i, model)
        mse_dict[i+1] = mse
    return mse, mse_dict

In [21]:
# Running the function for the start year and different models
slr_mse, slr_msedict = obtainCV(nC_data1.drop("enrollment", axis = 1), nC_data1["enrollment"],1989, 'slr')

mlr_mse, mlr_msedict = obtainCV(nC_data1.drop("enrollment", axis = 1), nC_data1["enrollment"],1989, 'mlr')

Once we obtain the MSE values, we then compare them side by side for different models

In [22]:
# Printing the performance of slr and mlr using mse
for i in range(1990,int(nC_data1['year'].max())+1):
    print(f"The SLR Cummulative MSE for year {i} is {slr_msedict[i]} and the MLR Cummulative MSE is {mlr_msedict[i]}")
    if slr_msedict[i] < mlr_msedict[i]:
        print(f"The Single Linear Regression function performed better for year {i}\n")
    elif slr_msedict[i] > mlr_msedict[i]:
        print(f"The Multivariate Linear Regression function performed better for year {i}\n")
    else:
        #extremely rare scenario
        print(f"Both the models performed the same\n")  

The SLR Cummulative MSE for year 1990 is 735386062957.1707 and the MLR Cummulative MSE is 519147666258.7235
The Multivariate Linear Regression function performed better for year 1990

The SLR Cummulative MSE for year 1991 is 1517659246647.7695 and the MLR Cummulative MSE is 1077427024497.0956
The Multivariate Linear Regression function performed better for year 1991

The SLR Cummulative MSE for year 1992 is 2347052423718.9766 and the MLR Cummulative MSE is 1669658294697.1494
The Multivariate Linear Regression function performed better for year 1992

The SLR Cummulative MSE for year 1993 is 3203280113551.451 and the MLR Cummulative MSE is 2283803322079.8213
The Multivariate Linear Regression function performed better for year 1993

The SLR Cummulative MSE for year 1994 is 4085932704898.1514 and the MLR Cummulative MSE is 2918988177436.3423
The Multivariate Linear Regression function performed better for year 1994

The SLR Cummulative MSE for year 1995 is 4997127260943.841 and the MLR Cu

Mean Squared Error (MSE) is a measure of the average squared difference between the predicted values and the actual values in a regression model. It is commonly used in machine learning and data science to evaluate the performance of a regression model, particularly in cases where the response variable is continuous.


The learnings we have from our MSE results are as follows:

* We can see from the above code block that the MSE values for **Multivariate Linear Regression** models are lesser than the MSE values from **Single Linear regression** models which means that the MLR model performs better for our data set.

* This is because of the extra predictor variables we have in the MLR model, the extra variables gives us a better fit for the data and thus the error is lesser in the difference between predicted and actual values.